# Ramen Clover training

In [2]:
#| default_exp 51-encoder-parallel-ramen-clover-for-wikititles

In [3]:
%load_ext autoreload
%autoreload 2

from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [4]:
#| export
import os,torch, torch.multiprocessing as mp, pickle
from xcai.basics import *
from xcai.models.PPP0XX import DBT017

In [5]:
os.environ['WANDB_MODE'] = 'disabled'

In [6]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5'
os.environ['WANDB_PROJECT']='xc-nlg_36-ramen-style-oak-training-pipeline-with-multitriplet-loss-with-clustering-for-wikititles'

In [12]:
#| export
data_dir = '/home/aiscuser/scratch/datasets'

In [14]:
block = XCBlock.from_cfg(data_dir, 'data_meta', dset='wikititles', valid_pct=0.001, tfm='rm', tokenizer='distilbert-base-uncased', 
                         smp_features=[('lbl2data|sal2lbl2data', 1, 1), ('sal2data', 1, 1)],
                         n_data_meta_samples=50, n_lbl_meta_samples=50)

/home/aiscuser/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [15]:
#| export
pkl_dir = f'{data_dir}/processed/'

In [16]:
with open(f'{pkl_dir}/wikititles_data-meta_distilbert-base-uncased_rm_ramen-sal-encoder-parallel.pkl', 'wb') as file: 
    pickle.dump(block, file)

In [10]:
#| export
with open(f'{pkl_dir}/wikititles_data-meta_distilbert-base-uncased_rm_ramen-sal-encoder-parallel.pkl', 'rb') as file: 
    block = pickle.load(file)

In [17]:
#| export
args = XCLearningArguments(
    output_dir='/home/aiscuser/outputs/51-encoder-parallel-ramen-clover-for-wikititles-1-0',
    logging_first_step=True,
    per_device_train_batch_size=600,
    per_device_eval_batch_size=100,
    representation_num_beams=200,
    representation_accumulation_steps=1,
    save_strategy="steps",
    evaluation_strategy='steps',
    eval_steps=1000,
    save_steps=1000,
    save_total_limit=5,
    num_train_epochs=300,
    predict_with_representation=True,
    adam_epsilon=1e-6,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-4,
    generation_num_beams=10,
    generation_length_penalty=1.5,
    predict_with_generation=True,
    representation_search_type='INDEX',
    group_by_cluster=True,
    num_clustering_warmup_epochs=10,
    num_cluster_update_epochs=5,
    num_cluster_size_update_epochs=10,
    clustering_type='EXPO',
    minimum_cluster_size=1,
    maximum_cluster_size=300,
    output_concatenation_weight=1.0,
    metric_for_best_model='P@1',
    load_best_model_at_end=True,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    fp16=True,
    label_names=['sal2data_idx', 'sal2data_input_ids', 'sal2data_attention_mask',
                 'sal2lbl2data_idx', 'sal2lbl2data_input_ids', 'sal2lbl2data_attention_mask'],
)

/opt/conda/envs/ptca/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
#| export
test_dset = block.test.dset.sample(n=2000, seed=50)
metric = PrecRecl(block.n_lbl, test_dset.data.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [19]:
#| export
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT017.from_pretrained('distilbert-base-uncased', ig_tok=0, bsz=bsz, tn_targ=5000, margin=0.3, tau=0.1, 
                               n_negatives=5, apply_softmax=True, lw=0.001, m_lw=0.3, meta_prefix='sal', 
                               tie_word_embeddings=False)
model.init_dr_head()

Some weights of DBT017 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['encoder.module.dr_layer_norm.bias', 'encoder.module.dr_layer_norm.weight', 'encoder.module.dr_projector.bias', 'encoder.module.dr_projector.weight', 'encoder.module.dr_transform.bias', 'encoder.module.dr_transform.weight', 'encoder.module.vocab_projector.weight', 'vocab_projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
#| export
trie = XCTrie.from_block(block)

  0%|          | 0/501070 [00:00<?, ?it/s]

In [21]:
#| export
learn = XCLearner(
    model=model, 
    args=args,
    trie=trie,
    train_dataset=block.train.dset,
    eval_dataset=test_dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
learn.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[2024-05-31 21:45:57,227] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: WARNING The `run_name

node-0:2603038:2603038 [0] NCCL INFO Bootstrap : Using eth0:10.13.60.215<0>
node-0:2603038:2603038 [0] NCCL INFO NET/Plugin : Plugin load (librccl-net.so) returned 2 : librccl-net.so: cannot open shared object file: No such file or directory
node-0:2603038:2603038 [0] NCCL INFO NET/Plugin : No plugin found, using internal implementation
node-0:2603038:2603038 [0] NCCL INFO Kernel version: 5.15.0-1042-azure
RCCL version 2.17.1+hip5.7 HEAD:cbbb3d8+

node-0:2603038:2605074 [0] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/misc/ibvwrap.cc:222 NCCL WARN Call to ibv_open_device failed

node-0:2603038:2605074 [0] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/transport/net_ib.cc:199 NCCL WARN NET/IB : Unable to open device mlx5_0

node-0:2603038:2605074 [0] /long_pathname_so_that_rpms_can_package_the_debug_info/src/extlibs/rccl/build/hipify/src/misc/ibvwrap.cc:222 NCCL WARN Call to ibv_open_device failed

nod

node-0:2603038:2605075 [1] NCCL INFO Ring 21 : 0 -> 1 -> 0 comm 0xc01cf10 nRanks 02 busId a00000
node-0:2603038:2605075 [1] NCCL INFO Ring 22 : 0 -> 1 -> 0 comm 0xc01cf10 nRanks 02 busId a00000
node-0:2603038:2605075 [1] NCCL INFO Ring 23 : 0 -> 1 -> 0 comm 0xc01cf10 nRanks 02 busId a00000
node-0:2603038:2605075 [1] NCCL INFO Trees [0] -1/-1/-1->1->0 [1] -1/-1/-1->1->0 [2] -1/-1/-1->1->0 [3] 0/-1/-1->1->-1 [4] 0/-1/-1->1->-1 [5] 0/-1/-1->1->-1 [6] -1/-1/-1->1->0 [7] -1/-1/-1->1->0 [8] -1/-1/-1->1->0 [9] 0/-1/-1->1->-1 [10] 0/-1/-1->1->-1 [11] 0/-1/-1->1->-1 [12] -1/-1/-1->1->0 [13] -1/-1/-1->1->0 [14] -1/-1/-1->1->0 [15] 0/-1/-1->1->-1 [16] 0/-1/-1->1->-1 [17] 0/-1/-1->1->-1 [18] -1/-1/-1->1->0 [19] -1/-1/-1->1->0 [20] -1/-1/-1->1->0 [21] 0/-1/-1->1->-1 [22] 0/-1/-1->1->-1 [23] 0/-1/-1->1->-1 comm 0xc01cf10 nRanks 02 busId a00000
node-0:2603038:2605074 [0] NCCL INFO Ring 13 : 1 -> 0 -> 1 comm 0xc098360 nRanks 02 busId 900000
node-0:2603038:2605074 [0] NCCL INFO Ring 14 : 1 -> 0 -> 1 co

/home/aiscuser/scratch/Projects/xcai/xcai/losses.py:21: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)
/opt/conda/envs/ptca/lib/python3.9/site-packages/transformers/utils/import_utils.py:533: FutureWarning: `is_torch_tpu_available` is deprecated and will be removed in 4.41.0. Please use the `is_torch_xla_available` instead.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
#| export
if __name__ == '__main__':
    mp.freeze_support()
    learn.train()